In [ ]:
from transformers import  AutoTokenizer, AutoModelForMaskedLM, pipeline
import json

In [2]:
device = 'cuda:0'
hug_token = ''
file_path = '../data/books/raw/forget.txt'
with open(file_path, 'r') as file:
    MUSE_dataset = file.read()

In [ ]:
# model_name_bert = "distilroberta-base"
model_name_bert = "bert-large-uncased"
tokenizer_bert = AutoTokenizer.from_pretrained(model_name_bert)
model_bert = AutoModelForMaskedLM.from_pretrained(model_name_bert)
fill_mask = pipeline("fill-mask", model=model_bert, tokenizer=tokenizer_bert, device=device)

In [5]:
def identify_common_words(sentence, pre_text_len, threshold=0.9):
    # Split sentence into words and track positions
    words = sentence.split()
    common_words = []

    # Get start and end positions for each word
    word_start_end_positions = []
    start_idx = 0
    for word in words:
        start_idx = sentence.find(word, start_idx)  # Find word's start index in the sentence
        end_idx = start_idx + len(word)             # Calculate end index
        word_start_end_positions.append((word, start_idx, end_idx))
        start_idx = end_idx                         # Update start_idx for the next search

    # Create masked sentences in a batch
    # masked_sentences = [sentence[:start] + "<mask>" + sentence[end:] for _, start, end in word_start_end_positions]
    masked_sentences = [sentence[:start] + "[MASK]" + sentence[end:] for _, start, end in word_start_end_positions]

    # Get predictions in a batch
    all_predictions = fill_mask(masked_sentences, top_k=1)

    # Identify non-common words based on predictions
    for i, (word, start, end) in enumerate(word_start_end_positions):
        # Check if the original word's probability is below the threshold
        if len(masked_sentences) == 1:
            all_predictions = [all_predictions]
        for pred in all_predictions[i]:
            try:
                if pred['token_str'].strip() == word and pred['score'] >= threshold:
                    common_words.append((word, start+pre_text_len, end+pre_text_len))
            except:
                print(pred)
                print(pred['token_str'].strip().lower())
                print(word.lower())

    return common_words

In [ ]:
import re
from tqdm import tqdm

all_common_words = None
common_words = []
pre_text_len = 0
splited_sentence =re.split(r'(\. |\n)', MUSE_dataset)

for item in tqdm(splited_sentence, desc="Processing sentences"):
    # print(item)
    output = identify_common_words(item, pre_text_len, threshold=0.9)
    common_words += output
    if item == '\n':
        pre_text_len += 1
    elif item == '. ':
        pre_text_len += 2
    else:
        pre_text_len += len(item)

all_common_words = [json.dumps({"word": item[0], "start": item[1], "end": item[2]}) for item in common_words]

In [24]:
save_path = "../data/books/raw/forget_common_words_bert.json"

with open(save_path, "w", encoding="utf-8") as f:
    json.dump(all_common_words, f, ensure_ascii=False, indent=4)